In [83]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 85)

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [84]:
def fetch_data(drop_some=True):

    df = pd.read_csv("abnormal_writeout.data.csv")

    if drop_some:
        # trascurare da ACC a UVM
        start_drop = df.columns.get_loc("ACC")
        end_drop = df.columns.get_loc("UVM")
        cols = np.arange(start_drop, end_drop + 1)
        df.drop(df.columns[cols], axis=1, inplace=True)

        # trascurare old_phylo_factor e la prima colonna
        df.drop("oldest_phylostratum_factor", axis=1, inplace=True)
        df.drop("Unnamed: 0", axis=1, inplace=True)

    return df

In [92]:
df = fetch_data()
df # 85 columns remain.

,response,cds_length,occ_total_sum,oldest_phylostratum,gc_cds,dnase_gene,dnase_cds,H3k4me1_gene,H3k4me3_gene,H3k27ac_gene,H3k4me1_cds,H3k4me3_cds,H3k27ac_cds,lamin_gene,repli_gene,nsome_gene,nsome_cds,transcription_gene,repeat_gene,repeat_cds,recomb_gene,AAA_freq,AAC_freq,AAG_freq,AAT_freq,ACA_freq,ACC_freq,ACG_freq,ACT_freq,AGA_freq,AGC_freq,AGG_freq,AGT_freq,ATA_freq,ATC_freq,ATG_freq,ATT_freq,CAA_freq,CAC_freq,CAG_freq,CAT_freq,CCA_freq,CCC_freq,CCG_freq,CCT_freq,CGA_freq,CGC_freq,CGG_freq,CGT_freq,CTA_freq,CTC_freq,CTG_freq,CTT_freq,GAA_freq,GAC_freq,GAG_freq,GAT_freq,GCA_freq,GCC_freq,GCG_freq,GCT_freq,GGA_freq,GGC_freq,GGG_freq,GGT_freq,GTA_freq,GTC_freq,GTG_freq,GTT_freq,TAA_freq,TAC_freq,TAG_freq,TAT_freq,TCA_freq,TCC_freq,TCG_freq,TCT_freq,TGA_freq,TGC_freq,TGG_freq,TGT_freq,TTA_freq,TTC_freq,TTG_freq,TTT_freq
0,0,-0.231676,-0.130244,1.901010,1.447504,1.190150,1.406066,0.022281,1.801757,-0.591137,0.472681,1.949397,-0.455623,-0.479855,0.577770,-0.254277,-0.303028,-0.093107,0.311667,-0.187624,-1.174934,-0.962971,-0.909913,-1.508122,-1.087512,-0.872773,1.602265,0.829956,-1.047882,-1.270695,1.036989,0.069038,-0.612966,-1.112302,-1.409381,-0.649559,-1.176062,-1.312306,0.391324,0.058646,-1.265710,0.384585,0.694620,1.409963,0.829268,1.797335,1.548229,0.903817,0.933719,-0.397981,-0.086160,1.414753,-0.346887,-1.243235,1.238157,0.943669,-0.824115,-0.356196,0.875956,1.696366,1.730229,0.054776,0.825449,1.145792,0.554808,-1.577040,0.447125,1.275182,-1.257541,-1.204038,-0.352835,-1.386701,-1.119012,-1.408061,0.539373,0.823120,-0.884699,0.161293,0.846379,0.539268,-0.628905,-1.111766,-0.365622,-1.157372,-0.412003
1,0,-0.479485,-0.225925,-0.683952,-1.218667,-0.855190,-0.428028,0.270554,-0.960246,-1.130393,-1.107698,-0.782452,-0.930966,2.172908,-0.728317,-0.208823,0.333337,-0.119886,-0.398704,-0.187624,0.584057,0.296533,1.149042,-0.065966,1.388289,1.197949,0.253892,-0.778624,-0.132201,1.166816,-1.541005,-0.402150,-1.145869,1.333728,0.803758,-0.693085,2.705568,0.155822,-0.765681,-0.887983,1.251116,-0.837808,-0.990130,-0.957495,-0.087426,-0.863659,-0.832224,-0.722976,-0.935078,-0.265076,0.174855,-0.599223,0.643374,1.017265,-0.225132,-0.370956,-0.282280,-0.689413,-1.277163,-0.978026,-0.930006,-0.676247,-0.838694,-0.038881,1.621418,-0.204153,-0.754478,-0.854915,0.639916,0.874857,1.208705,0.421527,1.247031,-0.065484,-0.327448,-0.151612,0.993725,0.200062,-0.895657,0.323194,-0.540768,2.205386,0.249155,1.918490,1.857967
2,1,-0.391241,-0.072836,-0.683952,0.482389,0.672705,0.928634,0.768924,0.019239,0.408770,0.695625,-0.699896,-0.723082,-0.479855,0.541861,0.772266,0.752547,-0.095956,0.083483,-0.187624,-0.427398,-0.104715,-0.181644,0.094350,-0.968550,0.069944,1.245460,0.097957,0.348878,0.087571,-0.020092,0.793797,-0.262386,-0.625139,-0.015744,0.063679,-0.847768,-0.097626,0.058707,0.230716,0.410751,1.302120,0.057532,-0.127896,0.624825,-0.388801,0.004821,0.294804,-0.559719,1.592316,-0.331791,0.390118,-0.739430,-0.674981,1.372832,0.379733,-0.626983,-0.528783,0.706419,-0.258796,0.598618,0.637801,0.690812,0.387350,0.448131,0.524565,-0.196243,-0.236192,-1.185901,-0.125129,0.573989,0.087986,-0.064738,-0.423978,-0.208042,0.784968,-1.144029,-0.571964,-0.053218,0.515054,-0.711053,-0.483593,-0.921655,-1.267833,-0.522241
3,0,0.296983,1.649415,-0.683952,-0.794498,-0.525282,-0.152416,0.003209,-0.815461,-0.437862,-0.461587,-0.723912,-0.569118,-0.479855,-1.137745,0.007207,0.798077,-0.111170,-0.178076,-0.187624,-0.190946,0.099659,0.298522,-0.005526,1.484987,0.713881,-0.454725,-0.193328,0.464364,0.145263,-0.500734,-0.893309,0.686977,0.905230,1.622440,0.609473,0.770784,1.164331,-0.696708,-0.068439,0.339796,-0.022302,-0.340180,-0.520962,-0.388488,-0.352416,-0.573491,-0.361055,-0.499743,-0.710679,-1.045508,-0.383306,0.040561,0.011920,1.428825,-0.734995,0.908960,-0.647483,-0.473808,-0.526752,-0.708639,-0.813613,-0.925574,-0.740244,0.185416,0.788469,-0.953331,0.313504,0.496344,1.057425,-0.315237,0.226605,1.158869,0.707721,-0.020972,-0.420635,-1.054114,1.616000,-0.07000

In [89]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [91]:
data = train_set.drop("response", axis=1)  # training set w/o labels
labels = train_set["response"].copy()  # labels
data

,cds_length,occ_total_sum,oldest_phylostratum,gc_cds,dnase_gene,dnase_cds,H3k4me1_gene,H3k4me3_gene,H3k27ac_gene,H3k4me1_cds,H3k4me3_cds,H3k27ac_cds,lamin_gene,repli_gene,nsome_gene,nsome_cds,transcription_gene,repeat_gene,repeat_cds,recomb_gene,AAA_freq,AAC_freq,AAG_freq,AAT_freq,ACA_freq,ACC_freq,ACG_freq,ACT_freq,AGA_freq,AGC_freq,AGG_freq,AGT_freq,ATA_freq,ATC_freq,ATG_freq,ATT_freq,CAA_freq,CAC_freq,CAG_freq,CAT_freq,CCA_freq,CCC_freq,CCG_freq,CCT_freq,CGA_freq,CGC_freq,CGG_freq,CGT_freq,CTA_freq,CTC_freq,CTG_freq,CTT_freq,GAA_freq,GAC_freq,GAG_freq,GAT_freq,GCA_freq,GCC_freq,GCG_freq,GCT_freq,GGA_freq,GGC_freq,GGG_freq,GGT_freq,GTA_freq,GTC_freq,GTG_freq,GTT_freq,TAA_freq,TAC_freq,TAG_freq,TAT_freq,TCA_freq,TCC_freq,TCG_freq,TCT_freq,TGA_freq,TGC_freq,TGG_freq,TGT_freq,TTA_freq,TTC_freq,TTG_freq,TTT_freq
13300,0.175294,0.118525,-0.448955,-0.945011,-0.954265,-0.791490,-0.969238,-0.913798,-0.921833,-0.706840,-0.719022,-0.530154,1.742704,-0.733096,-0.093588,0.756527,-0.119712,-0.230090,-0.187624,-0.439347,1.011802,1.359812,1.328939,0.475220,0.935366,-0.910335,-1.173391,0.602358,1.759545,-0.009164,-0.435713,1.080259,0.132863,1.654689,0.212061,0.434113,0.148035,-0.847332,1.260937,1.275854,-0.512488,-0.572756,-0.943306,-0.998354,-1.261749,-0.640511,-0.911815,-0.794583,0.421274,-0.289151,-0.659877,0.363680,1.800193,-0.074192,-0.694977,0.621173,0.308043,-1.018806,-0.627598,-0.643064,0.074813,-1.097190,-0.979161,-0.937861,-0.111330,0.164004,-1.041603,0.044562,0.598269,-0.736450,0.848652,0.008999,1.591273,-0.615150,-0.796999,0.837193,0.532584,-0.319801,-0.625726,-0.734367,0.245486,0.285616,0.484034,0.205866
15976,-0.218379,0.214206,-0.448955,-0.896574,-0.906374,-1.067059,-1.259419,-0.778248,-1.086890,-1.107698,-0.782452,-0.930966,-0.479855,0.305307,-0.070784,0.952663,-0.119933,0.526177,5.249018,-0.282281,0.856195,1.564466,-0.531522,2.269556,1.775385,0.386247,-0.665688,-0.281972,1.620325,-1.619972,0.085543,-0.531024,0.145000,1.323112,1.363707,-0.033826,-0.671348,1.326346,-0.193461,0.634490,-1.897710,-0.566064,-0.784450,0.212270,-0.460995,-0.816330,-1.003960,-0.502652,-0.134163,-0.153518,-1.313721,0.364127,2.337934,-1.299553,0.996900,0.323301,-1.179687,-1.089154,-0.660719,-1.497621,0.861177,-1.032914,-0.238461,-1.001279,0.748671,-1.432690,1.360930,-0.868948,0.319128,-0.099011,-0.172252,-0.040092,2.784178,-1.472052,-1.293949,-0.244860,1.073666,-0.904200,-0.167087,1.710238,-0.388501,0.242866,-0.209590,-0.057372
4069,-0.670479,-0.589511,1.901010,1.241097,0.363426,0.429239,1.151672,1.801757,1.356322,1.282143,1.949397,1.458597,-0.479855,0.645800,0.742975,0.544949,-0.119284,-0.451617,-0.187624,-1.088147,-1.255100,-1.464816,-0.724319,-0.813038,-1.464159,-0.260989,0.128925,0.927787,0.193197,-0.052473,1.476165,-2.028698,-1.220246,-1.106632,-0.086475,-1.429155,-1.587724,-0.184758,1.524292,-0.412676,0.172267,1.012029,0.387018,1.768584,-0.504348,0.924649,-0.669502,0.447451,0.239612,-0.781410,3.611991,-0.066344,-0.648563,0.516346,0.293236,-1.322691,2.249418,2.221575,0.243567,2.770548,-1.095668,1.044105,-0.231981,-0.772869,0.309354,-1.481646,-0.468575,-1.432688,-0.785181,0.114240,-0.050852,-1.321669,-1.192579,-0.840111,-0.977862,-1.260715,-0.187565,5.210743,-0.615234,-0.728566,-1.218392,-1.372600,-0.461679,-1.049092
7479,-0.101123,0.003709,-0.448955,-1.368561,-0.595264,-0.375685,-0.476991,-0.181827,-0.458217,-0.346694,-0.259300,-0.381050,-0.479855,-0.101504,0.061116,0.599643,-0.088464,0.119760,-0.187624,-0.292981,1.502725,0.866358,1.182712,1.715321,0.942791,-1.397733,-0.430665,-0.534509,1.036302,-0.297339,-1.032131,1.673783,1.365566,0.167677,0.517057,1.706812,1.244160,-0.374710,0.204160,0.893817,0.079666,-1.206862,-1.079173,-1.353048,-0.880549,-0.885908,-0.854920,-0.657713,-0.043077,-1.466234,-0.388349,-0.460444,0.948207,-1.210187,-1.111731,1.045870,0.437350,-1.021737,-0.762499,-0.286237,-0.883716,-1.007920,-1.075076,-1.348929,0.978663,0.446937,-1.142983,0.466323,1.936683,-0.550414,1.412060,0.687164,0.680840,0.046633,-1.055383,-0.072484,0.060851,

In [93]:
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),  # Remove null values
        ("std_scaler", StandardScaler()),  # Feature scaling
    ]
)

data_tr = pipeline.fit_transform(data)
data_prepared = pd.DataFrame(data_tr, columns=data.columns)

In [ ]:
def split_for_PCA(X):
    
    return